# Write LORS

In [3]:
using Pkg
Pkg.activate(normpath(joinpath(@__DIR__, "..")))  #activate environment in ,,/src

In [4]:
using Logging

In [5]:
using Plots
using DataFrames
using HDF5
using PhysicalConstants
using PhysicalConstants.CODATA2018
using Unitful
using Glob
using CSV
using Printf

In [6]:
using ATools

In [26]:
include("../src/ANema.jl") 

Main.ANema

In [8]:
import Unitful:
    nm, μm, mm, cm, ns, μs, ms, ps, s

In [9]:
function loglevel(log)
    if log == "Debug"
        logger = SimpleLogger(stdout, Logging.Debug)
    elseif log =="Info"
        logger = SimpleLogger(stdout, Logging.Info)
    else
        logger = SimpleLogger(stdout, Logging.Warn)
    end
    old_logger = global_logger(logger)
end

loglevel (generic function with 1 method)

In [10]:
loglevel("Warn")

Base.CoreLogging.SimpleLogger(IJulia.IJuliaStdio{Base.PipeEndpoint}(IOContext(Base.PipeEndpoint(RawFD(44) open, 0 bytes waiting))), Info, Dict{Any, Int64}())

## Read data file with nema events (merged data frames)

## Read mdf files

In [11]:
function pathfinder(rootdir, conf)
    mdf = string(conf, "-mdf")
    lor = string(conf, "-lor")
    phif = string("mdf-phistd-",conf,".csv")
    zf = string("mdf-zstd-",conf,".csv")

    mdfdir = joinpath(rootdir, mdf)
    phipath = joinpath(mdfdir, phif)
    zpath = joinpath(mdfdir, zf)
    
    lordir = joinpath(rootdir, lor)

    if isdir(lordir) == false
        mkdir(lordir)
    end

    philp = string("lor-phistd-",conf)
    zlp = string("lor-zstd-",conf)
    return (phipath= phipath, zpath = zpath, lordir=lordir, philp = philp, zlp = zlp)

end

pathfinder (generic function with 1 method)

In [12]:
function getlorpath(lordir=pathf.lordir, prefix=pathf.philp, posfix="first-true")
    fullpath = string(prefix,"-",posfix,".h5")
    joinpath(lordir, fullpath)
end

getlorpath (generic function with 4 methods)

In [13]:
dir = "../data/"
conf = "n3-w-20mm-all-qc2"
pathf = pathfinder(dir, conf)
println("pathfinder tuple =", pathf)

pathfinder tuple =(phipath = "../data/n3-w-20mm-all-qc2-mdf/mdf-phistd-n3-w-20mm-all-qc2.csv", zpath = "../data/n3-w-20mm-all-qc2-mdf/mdf-zstd-n3-w-20mm-all-qc2.csv", lordir = "../data/n3-w-20mm-all-qc2-lor", philp = "lor-phistd-n3-w-20mm-all-qc2", zlp = "lor-zstd-n3-w-20mm-all-qc2")


In [15]:
n3p = DataFrame(CSV.File(pathf.phipath))
n3z = DataFrame(CSV.File(pathf.zpath));


In [16]:
println(names(n3p))

["nsipm1", "nsipm2", "phistd1", "phistd2", "q1", "q2", "r1", "r2", "t1", "t2", "ta1", "ta2", "tr1", "tr2", "ux", "uy", "uz", "x1", "x2", "xb1", "xb2", "xr1", "xr2", "xs", "xt1", "xt2", "y1", "y2", "yb1", "yb2", "yr1", "yr2", "ys", "yt1", "yt2", "z1", "z2", "zb1", "zb2", "zr1", "zr2", "zs", "zstd1", "zstd2", "zt1", "zt2", "r1x", "r2x"]


In [17]:
println(names(n3z))

["nsipm1", "nsipm2", "phistd1", "phistd2", "q1", "q2", "r1", "r2", "t1", "t2", "ta1", "ta2", "tr1", "tr2", "ux", "uy", "uz", "x1", "x2", "xb1", "xb2", "xr1", "xr2", "xs", "xt1", "xt2", "y1", "y2", "yb1", "yb2", "yr1", "yr2", "ys", "yt1", "yt2", "z1", "z2", "zb1", "zb2", "zr1", "zr2", "zs", "zstd1", "zstd2", "zt1", "zt2", "r1x", "r2x"]


In [18]:
n3mp = ANema.setunits(n3p)
n3mz = ANema.setunits(n3z)

println("phi data frame has length of ", nrow(n3mp))
println("z data frame has length of ", nrow(n3mz))
println("phi data frame names ", names(n3mp))
println("z data frame names ", names(n3mz))

phi data frame has length of 132938
z data frame has length of 237131
phi data frame names ["nsipm1", "nsipm2", "phistd1", "phistd2", "zstd1", "zstd2", "q1", "q2", "r1", "r1x", "r2", "r2x", "t1", "t2", "ta1", "ta2", "tr1", "tr2", "ux", "uy", "uz", "x1", "x2", "xb1", "xb2", "xr1", "xr2", "xs", "xt1", "xt2", "y1", "y2", "yb1", "yb2", "yr1", "yr2", "ys", "yt1", "yt2", "z1", "z2", "zb1", "zb2", "zr1", "zr2", "zs", "zt1", "zt2"]
z data frame names ["nsipm1", "nsipm2", "phistd1", "phistd2", "zstd1", "zstd2", "q1", "q2", "r1", "r1x", "r2", "r2x", "t1", "t2", "ta1", "ta2", "tr1", "tr2", "ux", "uy", "uz", "x1", "x2", "xb1", "xb2", "xr1", "xr2", "xs", "xt1", "xt2", "y1", "y2", "yb1", "yb2", "yr1", "yr2", "ys", "yt1", "yt2", "z1", "z2", "zb1", "zb2", "zr1", "zr2", "zs", "zt1", "zt2"]


## Write LORS

In [19]:
lors_first_true = ANema.dftolor(n3mp, ANema.dtfirst, ANema.postrue);

In [20]:
lors_first_true[1]

Main.ANema.MlemLor{Float32}(-60.56419f0, 356.86978f0, 27.748032f0, 430.87402f0, -326.89825f0, 166.18393f0, 323.8185f0)

In [21]:
getlorpath(pathf.lordir,pathf.philp, "first-true")

"../data/n3-w-20mm-all-qc2-lor/lor-phistd-n3-w-20mm-all-qc2-first-true.h5"

In [28]:

ANema.write_lors_hdf5(getlorpath(pathf.lordir,pathf.philp, "first-true"), lors_first_true)

In [29]:
lors_first_reco = ANema.dftolor(n3mp, ANema.dtfirst, ANema.posreco)
ANema.write_lors_hdf5(getlorpath(pathf.lordir,pathf.philp, "first-reco"), lors_first_reco)

In [30]:
lors_minimum_reco = ANema.dftolor(n3mp, ANema.dtminimum, ANema.posreco)
ANema.write_lors_hdf5(getlorpath(pathf.lordir,pathf.philp, "minimum-reco"), lors_minimum_reco)

In [31]:
lors_minimum_recall = ANema.dftolor(n3mp, ANema.dtminimum, ANema.posrecall)
ANema.write_lors_hdf5(getlorpath(pathf.lordir,pathf.philp, "minimum-recall"), lors_minimum_recall)

In [32]:
lors_average_recall = ANema.dftolor(n3mp, ANema.dtaverage, ANema.posrecall)
ANema.write_lors_hdf5(getlorpath(pathf.lordir,pathf.philp, "average-recall"), lors_average_recall)

In [33]:
lors_first_true = ANema.dftolor(n3mz, ANema.dtfirst, ANema.postrue)
ANema.write_lors_hdf5(getlorpath(pathf.lordir,pathf.zlp, "first-true"), lors_first_true)
lors_first_reco = ANema.dftolor(n3mz, ANema.dtfirst, ANema.posreco)
ANema.write_lors_hdf5(getlorpath(pathf.lordir,pathf.zlp, "first-reco"), lors_first_reco)
lors_minimum_reco = ANema.dftolor(n3mz, ANema.dtminimum, ANema.posreco)
ANema.write_lors_hdf5(getlorpath(pathf.lordir,pathf.zlp, "minimum-reco"), lors_minimum_reco)
lors_minimum_recall = ANema.dftolor(n3mz, ANema.dtminimum, ANema.posrecall)
ANema.write_lors_hdf5(getlorpath(pathf.lordir,pathf.zlp, "minimum-recall"), lors_minimum_recall)
lors_average_recall = ANema.dftolor(n3mz, ANema.dtaverage, ANema.posrecall)
ANema.write_lors_hdf5(getlorpath(pathf.lordir,pathf.zlp, "average-recall"), lors_average_recall)